# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** My architecture is composed by an Encoder (that extracts information from the images), and a Decoder (which uses the output to generate a text description). For the encoder, I am using the pretrained version of ResNet.50, which was originally part of the Udacity document. For the decoder, I implemented the architecture described in [this paper](https://arxiv.org/pdf/1411.4555.pdf), which recommended a vocab_threshold=5 and an embed_size and hidden_size=512. The paper also recommended the use of dropout, however, I did not include dropout in my final configuration and I worked with just one layer, because I was focused on the release of a Minimum Viable Product for this experiment. Despite that, the generated results were fairly acceptable.


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** The original transform includes a RandomCrop and a RandomHorizontalFlip tranforms (which can help reduce overfitting) and a Resize and a Normalize tranforms (to standardize the input), which I believe are enough to preprocess the input to the format accepted by the pretrained network. For that reason, I left it as it was.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** I made two attempts, the first one included the ```params = list(decoder.parameters()) + list(encoder.embed.parameters()) + list(encoder.bn.parameters()) ```. I tried to include batch_normalization and dropout as part of my architecture, however, as I also selected a small batch_size of 32, I was experimenting a slow training time, and I was not getting a notable improvement on the accuracy. 

Then I decided to focus on the Minimum Viable Product, for which my second attempt included only the following trainable parameters: ```params = list(decoder.parameters()) + list(encoder.embed.parameters())```. This parameters are focused on optimize both the Encoder and the Decoder, which is the goal of this project. Also, reducing the number of trainable parameters improved the computational speed, which for this specific project, was more important to me.  


### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** I decided to use Adam optimizer as it is usually the most widely used  optimizer for deep learning and usually delivers a better performance than other optimizers such as SGD in terms of speed and accuracy. Among its properties, I can mention an adaptative learning rate, which adds value to the experiment.

In [4]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 128          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 256           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3            # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...




  0%|          | 0/414113 [00:00<?, ?it/s]

  0%|          | 377/414113 [00:00<01:49, 3764.71it/s]

  0%|          | 846/414113 [00:00<01:43, 3999.68it/s]

  0%|          | 1327/414113 [00:00<01:38, 4210.59it/s]

  0%|          | 1809/414113 [00:00<01:34, 4374.49it/s]

  1%|          | 2266/414113 [00:00<01:32, 4429.43it/s]

  1%|          | 2727/414113 [00:00<01:31, 4480.38it/s]

  1%|          | 3196/414113 [00:00<01:30, 4540.20it/s]

  1%|          | 3661/414113 [00:00<01:29, 4572.28it/s]

  1%|          | 4142/414113 [00:00<01:28, 4640.67it/s]

  1%|          | 4627/414113 [00:01<01:27, 4700.71it/s]

  1%|          | 5114/414113 [00:01<01:26, 4749.95it/s]

  1%|▏         | 5594/414113 [00:01<01:25, 4763.42it/s]

  1%|▏         | 6066/414113 [00:01<01:26, 4735.39it/s]

  2%|▏         | 6537/414113 [00:01<01:26, 4700.79it/s]

  2%|▏         | 7005/414113 [00:01<01:29, 4527.83it/s]

  2%|▏         | 7478/414113 [00:01<01:28, 4583.74it/s]

  2%|▏         | 7958/414113 [00:01<01:27, 4

 30%|██▉       | 122907/414113 [00:26<01:00, 4802.57it/s]

 30%|██▉       | 123388/414113 [00:26<01:00, 4772.87it/s]

 30%|██▉       | 123866/414113 [00:26<01:00, 4772.73it/s]

 30%|███       | 124357/414113 [00:26<01:00, 4812.97it/s]

 30%|███       | 124839/414113 [00:26<01:00, 4812.30it/s]

 30%|███       | 125324/414113 [00:26<00:59, 4821.58it/s]

 30%|███       | 125812/414113 [00:26<00:59, 4838.52it/s]

 30%|███       | 126296/414113 [00:26<00:59, 4831.86it/s]

 31%|███       | 126781/414113 [00:26<00:59, 4836.92it/s]

 31%|███       | 127265/414113 [00:26<00:59, 4829.15it/s]

 31%|███       | 127748/414113 [00:27<00:59, 4790.11it/s]

 31%|███       | 128233/414113 [00:27<00:59, 4805.36it/s]

 31%|███       | 128715/414113 [00:27<00:59, 4808.42it/s]

 31%|███       | 129201/414113 [00:27<00:59, 4822.94it/s]

 31%|███▏      | 129699/414113 [00:27<00:58, 4868.28it/s]

 31%|███▏      | 130196/414113 [00:27<00:57, 4897.45it/s]

 32%|███▏      | 130686/414113 [00:27<00:57, 4890.28it/s

 59%|█████▉    | 244798/414113 [00:51<00:35, 4798.34it/s]

 59%|█████▉    | 245284/414113 [00:51<00:35, 4815.65it/s]

 59%|█████▉    | 245766/414113 [00:52<00:35, 4803.89it/s]

 59%|█████▉    | 246252/414113 [00:52<00:34, 4818.31it/s]

 60%|█████▉    | 246734/414113 [00:52<00:34, 4798.37it/s]

 60%|█████▉    | 247216/414113 [00:52<00:34, 4802.66it/s]

 60%|█████▉    | 247697/414113 [00:52<00:34, 4790.49it/s]

 60%|█████▉    | 248177/414113 [00:52<00:34, 4779.62it/s]

 60%|██████    | 248658/414113 [00:52<00:34, 4787.09it/s]

 60%|██████    | 249137/414113 [00:52<00:34, 4782.73it/s]

 60%|██████    | 249616/414113 [00:52<00:34, 4760.26it/s]

 60%|██████    | 250099/414113 [00:52<00:34, 4779.16it/s]

 61%|██████    | 250580/414113 [00:53<00:34, 4787.69it/s]

 61%|██████    | 251065/414113 [00:53<00:33, 4806.13it/s]

 61%|██████    | 251551/414113 [00:53<00:33, 4821.68it/s]

 61%|██████    | 252045/414113 [00:53<00:33, 4855.22it/s]

 61%|██████    | 252533/414113 [00:53<00:33, 4859.38it/s

 89%|████████▊ | 366710/414113 [01:17<00:09, 4749.63it/s]

 89%|████████▊ | 367186/414113 [01:17<00:20, 2317.53it/s]

 89%|████████▉ | 367652/414113 [01:17<00:17, 2728.57it/s]

 89%|████████▉ | 368135/414113 [01:17<00:14, 3137.30it/s]

 89%|████████▉ | 368613/414113 [01:18<00:13, 3497.76it/s]

 89%|████████▉ | 369096/414113 [01:18<00:11, 3811.43it/s]

 89%|████████▉ | 369574/414113 [01:18<00:10, 4057.68it/s]

 89%|████████▉ | 370042/414113 [01:18<00:10, 4225.36it/s]

 89%|████████▉ | 370510/414113 [01:18<00:10, 4349.54it/s]

 90%|████████▉ | 370974/414113 [01:18<00:09, 4432.64it/s]

 90%|████████▉ | 371438/414113 [01:18<00:09, 4480.31it/s]

 90%|████████▉ | 371902/414113 [01:18<00:09, 4524.79it/s]

 90%|████████▉ | 372371/414113 [01:18<00:09, 4571.58it/s]

 90%|█████████ | 372846/414113 [01:18<00:08, 4621.59it/s]

 90%|█████████ | 373329/414113 [01:19<00:08, 4681.49it/s]

 90%|█████████ | 373805/414113 [01:19<00:08, 4703.60it/s]

 90%|█████████ | 374293/414113 [01:19<00:08, 4754.85it/s

Done (t=0.87s)
creating index...
index created!
Obtaining caption lengths...


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [5]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/3236], Loss: 4.2605, Perplexity: 70.84232
Epoch [1/3], Step [200/3236], Loss: 3.3627, Perplexity: 28.8670
Epoch [1/3], Step [300/3236], Loss: 3.2730, Perplexity: 26.39093
Epoch [1/3], Step [400/3236], Loss: 2.9938, Perplexity: 19.9611
Epoch [1/3], Step [500/3236], Loss: 3.5221, Perplexity: 33.8564
Epoch [1/3], Step [600/3236], Loss: 3.1177, Perplexity: 22.5934
Epoch [1/3], Step [700/3236], Loss: 2.7974, Perplexity: 16.4018
Epoch [1/3], Step [800/3236], Loss: 2.8968, Perplexity: 18.1167
Epoch [1/3], Step [900/3236], Loss: 2.5884, Perplexity: 13.3086
Epoch [1/3], Step [1000/3236], Loss: 2.6568, Perplexity: 14.2506
Epoch [1/3], Step [1100/3236], Loss: 2.5247, Perplexity: 12.4866
Epoch [1/3], Step [1200/3236], Loss: 2.5221, Perplexity: 12.4545
Epoch [1/3], Step [1300/3236], Loss: 2.6884, Perplexity: 14.70822
Epoch [1/3], Step [1400/3236], Loss: 2.4946, Perplexity: 12.1174
Epoch [1/3], Step [1500/3236], Loss: 2.3663, Perplexity: 10.6579
Epoch [1/3], Step [1600/3236], 

In [ ]:
1

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.